<span type="title">Filter and Wrapper</span> | <span type="update">2018-09-26</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Servlet 和 JSP 应用程序的请求过滤问题。采用 Filter 过滤器类可以方便的对于 请求、包含等动作进行监听和处理，用于审计、日志、安全审查等方面。</p></span>

# 过滤器

## 实现过滤器类

过滤器用来劫持对于 Servlet 的请求，并且可以重写 Servlet 的相应结果。我们一般使用过滤器进行安全审查、日志和审计工作、压缩相应流等等。Servlet 并不会知道传递给自己的 req 或者自己写好的 resp 被更改了。

过滤器使用了装饰者设计模式，一个过滤器类必须实现 Servlet 的 Filter 接口，然后实现 init、doFilter 以及 destroy 方法，其中可供调用的对象是 ServletContext 以及 FilterConfig 对象。使用者必须在 init 方法中创建类实例来保存着两个对象（大部分情况下）。doFilter 会接受 req，resp 以及 FilterChain 三个对象，对于前两者需要转型为 HttpServletRequest/Response。FilterChain 的作用在于指示此过滤器传递给下一个过滤器来执行进一步的动作。

举例如下，这个例子的过滤器获取 request 对象，打印日志，然后交给下一个 Filter，最后的 Filter 交给 Servlet 进行响应。此外需要注意，doFilter 需要能够抛出 ServletException 以及 IOException 这两个错误。

```java
public class UserFilter implements Filter {
    private FilterConfig config;
    private ServletContext context;
    @Override
    public void doFilter(ServletRequest servletRequest, ServletResponse servletResponse, FilterChain filterChain)
                throws ServletException, IOException {
        HttpServletRequest request = (HttpServletRequest) servletRequest;
        HttpServletResponse response = (HttpServletResponse) servletResponse;
        String name;
        if ((name = request.getRemoteUser()) != null) 
            context.log("User" + name + "is updating...");
        filterChain.doFilter(request,response);
    }

    @Override
    public void init(FilterConfig filterConfig) {
        this.config = filterConfig;
        this.context = config.getServletContext();
        context.log(config.getFilterName() + "is init now...");
    }

    @Override
    public void destroy() {
        context.log(config.getFilterName() + "is destroy now...");
    }
}
```

## 确定过滤器链

在 DD 中进行如下设置：在 filter 标签中设置过滤器名称以及对应的过滤器类，还包括可选的参数信息。然后在 mapping 中设置此过滤器对应的 url 模式匹配或者 servlet 匹配。两者必须有一个。如果一个 url 符合多个匹配，那么按照 DD 的顺序设置 chain。如果对于一个 Servlet，有 url 和 Servlet 两种指定的过滤器，那么 url 指定的过滤器放在前面，servlet 指定的过滤器放在后面（更靠近Servlet）。

在 mapping 中，还可以设置对于不同的请求方式进行过滤，默认为 request，可选 include、forward、error。

```xml
<filter>
    <filter-name>userfilter</filter-name>
    <filter-class>com.mazhangjing.filter.UserFilter</filter-class>
    <init-param>
        <param-name>namespace</param-name>
        <param-value>hi</param-value>
    </init-param>
</filter>
<filter-mapping>
    <filter-name>userfilter</filter-name>
    <url-pattern>*.jsp</url-pattern> <!--OR <servlet-name>HelloAgain</servlet-name> -->
    <dispatcher>REQUEST</dispatcher> <!-- OR -->
    <dispatcher>INCLUDE</dispatcher> <!-- OR -->
    <dispatcher>FORWARD</dispatcher> <!-- OR -->
    <dispatcher>ERROR</dispatcher> <!-- OR -->
</filter-mapping>
```

然后请求一个 .jsp 的页面，可以看到这个 Filter 工作了：

```
userfilter is init now...
[2018-09-26 05:13:31,409] Artifact JSPLearn:war exploded: Artifact is deployed successfully
[2018-09-26 05:13:31,409] Artifact JSPLearn:war exploded: Deploy took 1,855 milliseconds
Filter work now...
[2018-09-26 05:13:45,924] Artifact JSPLearn:war exploded: Artifact is being deployed, please wait...
userfilter is destroy now...
```

一个过滤器可以进行请求派发，而不是继续传递请求到下一个链的对象，比如登录认证失败。或者进行URL跳转，跳转到某个指定的URL上去。

# 包装器

过滤器总是在 Servlet 处理请求之间工作吗？不一定，我们可以在过滤器的 `filterChain.doFilter(req,resp)` 之后写下代码，这些代码总是在Servlet响应后才执行的。但问题是，在这个时候，Servlet已经将 Response 返回给了客户，因此我们不能对于 response 进行进一步的处理。这时候，就需要使用 HttpResponseWrapper 了，这是一个使用了包装器设计模式的 HttpResponse 接口的类。我们可以使用此返回给 Servlet，然后在 doFilter 调用返回后进行进一步的操作。

```java
public void doFilter(ServletRequest servletRequest, ServletResponse servletResponse, FilterChain filterChain) 
                            throws ServletException,IOException {
    if (((HttpServletRequest)servletRequest)
            .getHeader("Accept-Encoding").indexOf("gzip") > -1) {
        CompressResponseWrapper wrapper = 
                new CompressResponseWrapper((HttpServletResponse) servletResponse);
        wrapper.setHeader("Context-Encoding","gzip");
        filterChain.doFilter(servletRequest,wrapper);
        GZIPOutputStream gzos = wrapper.getGZIPOutputStream();
        gzos.finish(); //GZIP必须在处理完后关闭
        System.out.println("Finished handle gzip response");
    } else {
        System.out.println("No handle gzip response");
        filterChain.doFilter(servletRequest,servletResponse);
    }

}
```

一个包装器用来过滤 response 的例子如上所示。我们截获了 response 流，然后进行了包装和压缩。